<a href="https://colab.research.google.com/github/yutan0565/colab_git/blob/main/code/MobileNet_V3_Small_%EC%98%88%EC%8B%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##기본 모델 형성

In [1]:
!pip install -q tensorflow-model-optimization

     |████████████████████████████████| 237 kB 16.3 MB/s 


In [2]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import pathlib
import os
import datetime

from tensorflow.keras.applications import MobileNetV3Small
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow import keras
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, CSVLogger, TensorBoard
import tensorflow_model_optimization as tfmot

In [3]:
(raw_train_x, raw_train_y), (raw_test_x, raw_test_y) = tf.keras.datasets.cifar10.load_data()

"""
print(raw_train_x.shape)
print(raw_test_x.shape)
print(raw_train_y.shape)
print(raw_test_y.shape)
"""

170508288/170498071 [==============================] - 2s 0us/step


'\nprint(raw_train_x.shape)\nprint(raw_test_x.shape)\nprint(raw_train_y.shape)\nprint(raw_test_y.shape)\n'

In [4]:
# float을 넣을거면 0~1 사이 값으로 바꿔야함
# integer 형태로 넣어보기
train_x = raw_train_x[:45000].astype(np.float32)/255.0
valid_x = raw_train_x[45000:].astype(np.float32)/255.0
test_x = raw_test_x.astype(np.float32)/255.0


train_y = raw_train_y[:45000]
valid_y = raw_train_y[45000:]
test_y = raw_test_y
labels = ["airplane", "automobile", "bird", "cat", "deer", "dog", "frog", "horse", "ship", "truck"]


"""
print(train_x.shape)
print(valid_x.shape)
print(test_x.shape)
print(train_y.shape)
print(valid_y.shape)
print(test_y.shape)

def show_sample(i):
  print(raw_train_y[i][0], labels[raw_train_y[i][0]])
  plt.imshow(raw_train_x[i])
  plt.show()

for i in [2, 10, 12, 14]:
  show_sample(i)
"""

'\nprint(train_x.shape)\nprint(valid_x.shape)\nprint(test_x.shape)\nprint(train_y.shape)\nprint(valid_y.shape)\nprint(test_y.shape)\n\ndef show_sample(i):\n  print(raw_train_y[i][0], labels[raw_train_y[i][0]])\n  plt.imshow(raw_train_x[i])\n  plt.show()\n\nfor i in [2, 10, 12, 14]:\n  show_sample(i)\n'

In [5]:
mobile = MobileNetV3Small(  #weights = 'imagenet',  #그냥 초기화 하는거면, 이거 지우기
                            include_top = False,
                            input_shape=(32,32,3)
                            )

# vgg conv 구조만 사용하고 마지막 FC layer는 다른거 사용

fc_layer = keras.Sequential([
                             layers.Flatten(),
                             layers.Dense(1024, activation = 'relu'),
                             layers.Dense(1024, activation = 'relu'),
                             layers.Dense(1024, activation = 'relu'),
                             layers.Dense(10, activation = "sigmoid")
                             ])

model = keras.Sequential([mobile,
                          fc_layer
                          ])
model.summary()
# mobile.summary()
# fc_layer.summary()

4349952/4334752 [==============================] - 0s 0us/step
Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 MobilenetV3small (Functiona  (None, 1, 1, 576)        939120    
 l)                                                              
                                                                 
 sequential (Sequential)     (None, 10)                2700298   
                                                                 
Total params: 3,639,418
Trainable params: 3,627,306
Non-trainable params: 12,112
_________________________________________________________________


In [9]:
# Callback 함수 지정 해주기
early_stop = EarlyStopping(patience=30) 
mc = ModelCheckpoint("./best_model/mobile_original_checkpoint", 
                     save_best_only=True,
                     verbose = 1,
                     mode = 'auto') 
reduce_lr  = ReduceLROnPlateau(monitor = 'val_loss',
                               factor=0.5, 
                               patience=5
                               ) 
# tensorboard 관련 
def make_Tensorboard_dir(dir_name):
  root_logdir = os.path.join(os.curdir, dir_name)
  sub_dir_name = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
  return os.path.join(root_logdir, sub_dir_name)

dir_name = "Learning_log"
TB_log_dir = make_Tensorboard_dir(dir_name)
TensorB = TensorBoard(log_dir = TB_log_dir)


#optimizaer  조정 해주기
opt = tf.keras.optimizers.Adam(learning_rate=0.0001)

In [ ]:
# optimizer, loss 함수를 정의하고,  학습 준비를 한다,  metrics 는 어떤 일이 발생하는지 보여줄 것들
model.compile(optimizer=opt, loss="sparse_categorical_crossentropy", metrics=["accuracy"])



# 한번에 몇개의 데이터 학습하고 가중치 갱신할지 
model.fit(train_x, train_y,
          epochs=100,
          verbose=1,
          batch_size=32,
          #validation_split = 0.1
          validation_data = (valid_x, valid_y),
          callbacks = [early_stop, reduce_lr , mc]
          )

Epoch 1/100
1407/1407 [==============================] - ETA: 0s - loss: 0.3569 - accuracy: 0.8715

INFO:tensorflow:Assets written to: ./best_model/mobile_original_checkpoint/assets


INFO:tensorflow:Assets written to: ./best_model/mobile_original_checkpoint/assets


1407/1407 [==============================] - 199s 138ms/step - loss: 0.3569 - accuracy: 0.8715 - val_loss: 2.8450 - val_accuracy: 0.4390 - lr: 1.0000e-04
Epoch 2/100
1407/1407 [==============================] - 159s 113ms/step - loss: 0.3434 - accuracy: 0.8748 - val_loss: 5.0730 - val_accuracy: 0.3078 - lr: 1.0000e-04
Epoch 3/100
1407/1407 [==============================] - 156s 111ms/step - loss: 0.3350 - accuracy: 0.8799 - val_loss: 19.5059 - val_accuracy: 0.1496 - lr: 1.0000e-04
Epoch 4/100
1407/1407 [==============================] - ETA: 0s - loss: 0.3318 - accuracy: 0.8784

INFO:tensorflow:Assets written to: ./best_model/mobile_original_checkpoint/assets


INFO:tensorflow:Assets written to: ./best_model/mobile_original_checkpoint/assets


1407/1407 [==============================] - 178s 127ms/step - loss: 0.3318 - accuracy: 0.8784 - val_loss: 2.4083 - val_accuracy: 0.4672 - lr: 1.0000e-04
Epoch 5/100
1407/1407 [==============================] - ETA: 0s - loss: 0.3384 - accuracy: 0.8807

INFO:tensorflow:Assets written to: ./best_model/mobile_original_checkpoint/assets


INFO:tensorflow:Assets written to: ./best_model/mobile_original_checkpoint/assets


1407/1407 [==============================] - 185s 132ms/step - loss: 0.3384 - accuracy: 0.8807 - val_loss: 1.5125 - val_accuracy: 0.6332 - lr: 1.0000e-04
Epoch 6/100
1062/1407 [=====================>........] - ETA: 40s - loss: 0.3396 - accuracy: 0.8763

In [ ]:
loss, acc = model.evaluate(test_x, test_y)
print("loss=",loss)
print("acc=",acc)

y_ = model.predict(test_x)
predicted = np.argmax(y_, axis=1)

print(predicted)

In [ ]:
model.save("./saved_model/mobile_original.h5")
model.save("./saved_model/mobile_original_checkpoint")

##int_8 Quantization 진행

In [ ]:
# 구조까지 들어가 있는거
model = tf.keras.models.load_model('./saved_model/mobile_original_checkpoint')

In [ ]:
# input 에 대해서 변수 설정
def representative_data_gen():
  for input_value in tf.data.Dataset.from_tensor_slices(train_x).batch(1).take(100):
    yield [input_value]

In [ ]:
# int 8 로 quantization 진행하기
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = representative_data_gen

# Ensure that if any ops can't be quantized, the converter throws an error
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]

# Set the input and output tensors to uint8 (APIs added in r2.3)
converter.inference_input_type = tf.uint8
converter.inference_output_type = tf.uint8

tflite_model_quant = converter.convert()

In [ ]:
# 그냥 파일 형태만 tflite로 변환 (float 형태임)
converter = tf.lite.TFLiteConverter.from_keras_model(model)

tflite_model = converter.convert()

In [ ]:
interpreter = tf.lite.Interpreter(model_content=tflite_model_quant)
input_type = interpreter.get_input_details()[0]['dtype']
print('input: ', input_type)
output_type = interpreter.get_output_details()[0]['dtype']
print('output: ', output_type)

In [ ]:
# 모델 저장 해주는 과정
import pathlib

tflite_models_dir = pathlib.Path("/tmp/mobile_tflite_models/")
tflite_models_dir.mkdir(exist_ok=True, parents=True)

# Save the unquantized/float model:
tflite_model_file = tflite_models_dir/"mobile_model.tflite"
tflite_model_file.write_bytes(tflite_model)
# Save the quantized model:
tflite_model_quant_file = tflite_models_dir/"mobiile_model_quant.tflite"
tflite_model_quant_file.write_bytes(tflite_model_quant)

In [ ]:
def run_tflite_model(tflite_file, test_image_indices):
  global test_x

  # Initialize the interpreter
  interpreter = tf.lite.Interpreter(model_path=str(tflite_file))
  interpreter.allocate_tensors()

  input_details = interpreter.get_input_details()[0]
  output_details = interpreter.get_output_details()[0]

  predictions = np.zeros((len(test_image_indices),), dtype=int)
  for i, test_image_index in enumerate(test_image_indices):
    test_image = test_x[test_image_index]
    test_label = test_y[test_image_index]

    # Check if the input type is quantized, then rescale input data to uint8
    if input_details['dtype'] == np.uint8:
      input_scale, input_zero_point = input_details["quantization"]
      test_image = test_image / input_scale + input_zero_point

    test_image = np.expand_dims(test_image, axis=0).astype(input_details["dtype"])
    interpreter.set_tensor(input_details["index"], test_image)
    interpreter.invoke()
    output = interpreter.get_tensor(output_details["index"])[0]

    predictions[i] = output.argmax()

  return predictions

In [ ]:
def evaluate_model(tflite_file, model_type):
  global test_x
  global test_y

  test_image_indices = range(test_x.shape[0])
  predictions = run_tflite_model(tflite_file, test_image_indices)

  accuracy = (np.sum(test_y.reshape(-1)== predictions) * 100) / len(test_x)

  print('%s model accuracy is %.4f%% (Number of test samples=%d)' % (
      model_type, accuracy, len(test_x)))

In [ ]:
import matplotlib.pylab as plt

# Change this to test a different image
test_image_index = 1

## Helper function to test the models on one image
def test_model(tflite_file, test_image_index, model_type):
  global test_labels

  predictions = run_tflite_model(tflite_file, [test_image_index])

  plt.imshow(test_x[test_image_index])
  template = model_type + " Model \n True:{true}, Predicted:{predict}"
  _ = plt.title(template.format(true= str(test_y[test_image_index]), predict=str(predictions[0])))
  plt.grid(False)

In [ ]:
test_model(tflite_model_file, test_image_index, model_type="Float")

In [ ]:
test_model(tflite_model_quant_file, test_image_index, model_type="Quantized")

##Pruning 진행

In [ ]:
_, baseline_model_accuracy = model.evaluate(
    test_x, test_y, verbose=0)

print('Baseline test accuracy:', baseline_model_accuracy)

_, keras_file = tempfile.mkstemp('.h5')
tf.keras.models.save_model(model, keras_file, include_optimizer=False)
print('Saved baseline model to:', keras_file)

In [ ]:
class MyDenseLayer(tf.keras.layers.Dense, tfmot.sparsity.keras.PrunableLayer):

  def get_prunable_weights(self):
    # Prune bias also, though that usually harms model accuracy too much.
    return [self.kernel, self.bias]

# Use `prune_low_magnitude` to make the `MyDenseLayer` layer train with pruning.
model_for_pruning = tf.keras.Sequential([
  tfmot.sparsity.keras.prune_low_magnitude(MyDenseLayer(20, input_shape=input_shape)),
  tf.keras.layers.Flatten()
])

model_for_pruning.summary()

In [ ]:
prune_low_magnitude = tfmot.sparsity.keras.prune_low_magnitude

# Compute end step to finish pruning after 2 epochs.
batch_size = 128
epochs = 2
validation_split = 0.1 # 10% of training set will be used for validation set. 

num_images = train_x.shape[0] * (1 - validation_split)
end_step = np.ceil(num_images / batch_size).astype(np.int32) * epochs


# Define model for pruning.
pruning_params = {
      'pruning_schedule': tfmot.sparsity.keras.PolynomialDecay(initial_sparsity=0.50,
                                                               final_sparsity=0.80,
                                                               begin_step=0,
                                                               end_step=end_step)
}

model_for_pruning = prune_low_magnitude(model, **pruning_params)

# `prune_low_magnitude` requires a recompile.
model_for_pruning.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

model_for_pruning.summary()

In [ ]:
logdir = tempfile.mkdtemp()

callbacks = [
  tfmot.sparsity.keras.UpdatePruningStep(),
  tfmot.sparsity.keras.PruningSummaries(log_dir=logdir),
]

model_for_pruning.fit(train_x, train_,
                  batch_size=batch_size, epochs=epochs, validation_split=validation_split,
                  callbacks=callbacks)

In [ ]:
_, model_for_pruning_accuracy = model_for_pruning.evaluate(
   test_x, test_y, verbose=0)

print('Baseline test accuracy:', baseline_model_accuracy) 
print('Pruned test accuracy:', model_for_pruning_accuracy)

In [ ]:
%tensorboard --logdir={logdir}

In [ ]:
"""
1.	경량 딥러닝 기술 배경
  1.	 기술 배경
  2.	기술 동향
2.	경량 알고리즘
  1.	모델 구조
  2.	합성곱 필터 
  3.	자동 모델 탐색
3.	알고리즘 경량화
  1.	모델 압축
  2.	지식 증류
  3.	하드웨어 가속화
  4.	모델 암축 자동 탐색
4.	실험
  1.	비교 모델
  2.	개발 환경 및 code 구성
  3.	성능 비교
  4.	Insight
5.	차주 계획

"""